In [12]:
import random
def get_data(dtype="training"):
    fp = open("digitdata/"+dtype+"images", "r")
    count = 0
    data_point = [[0 for _ in range(28)] for _ in range(28)]
    data = []
    for a in fp:
        count += 1
        if count == 1:
            data_point = [[0 for _ in range(28)] for _ in range(28)]
            line = a.replace("\n","")
            if "+" in line or "#" in line:
                c  = 0
                for i in str(line):
                    if i == '+' or i == '#':
                        data_point[count-1][c] = 1
                    c += 1
        elif count == 28: 
            line = a.replace("\n","")
            if "+" in line or "#" in line:
                c  = 0 
                for i in str(line):
                    if i == '+' or i == '#':
                        data_point[count-1][c] = 1
                    c += 1
            count = 0
            data.append(data_point)
        else:
            line = a.replace("\n","")
            if "+" in line or "#" in line:
                c  = 0 
                for i in str(line):
                    if i == '+' or i == '#':
                        data_point[count-1][c] = 1
                    c += 1        
    fp.close()
    actual_y = []
    fp = open("digitdata/"+dtype+"labels","r")
    for a in fp:
        actual_y.append(int(a.replace("\n","")))
    fp.close()
    

    return data, actual_y




In [39]:
columns = ["Data%"]

for i in range(1,51):
    columns.append('Prediction_Error_Test_Trail_'+str(i))

for i in range(1,51):
    columns.append('Prediction_Error_Validation_Trail_'+str(i))

for i in range(1,51):
    columns.append('Training_Time_Trail_'+str(i))
df = pd.DataFrame(columns = columns)

df['Data%'] = [x for x in range(10,101, 10)]
df = df.set_index('Data%')
#df.loc[20]['Correct_Predictions_Test_Trail_1'] = 333
df

,Prediction_Error_Test_Trail_1,Prediction_Error_Test_Trail_2,Prediction_Error_Test_Trail_3,Prediction_Error_Test_Trail_4,Prediction_Error_Test_Trail_5,Prediction_Error_Test_Trail_6,Prediction_Error_Test_Trail_7,Prediction_Error_Test_Trail_8,Prediction_Error_Test_Trail_9,Prediction_Error_Test_Trail_10,...,Training_Time_Trail_41,Training_Time_Trail_42,Training_Time_Trail_43,Training_Time_Trail_44,Training_Time_Trail_45,Training_Time_Trail_46,Training_Time_Trail_47,Training_Time_Trail_48,Training_Time_Trail_49,Training_Time_Trail_50
Data%,,,,,,,,,,,,,,,,,,,,,
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:

import sys
import pandas as pd
import random
import time
training_data, training_y = get_data() 
for p in range(10, 101, 10):
    
    training_percent = p
    
    for trail in range(1,51):
        print('Percentage',p,'Trail',trail, 'Started')
        #print("\n////////////////////////////////")
        #print("// ⦿ training data used:", str(training_percent)+ "%")


        for i in range(len(training_data)):
            training_data[i].append(training_y[i])

        random.shuffle(training_data)

        training_y = []
        for i in range(len(training_data)):
            training_y.append(training_data[i].pop())
            
        
        st = time.time()

        features = []

        #inos = random.sample([x for x in range(5000)], 50*training_percent)
        inos = [x for x in range(50*training_percent)]
        prob_table_digits = [[[[0 for _ in range(17)] for _ in range(7)] for _ in range(7)] for _ in range(10)]
        num_digits = [0 for _ in range(10)]

        for ino in inos:
            feature = [[0 for _ in range(7)] for _ in range(7)]
            for x in range(7):
                for y in range(7):
                    i = 4*x
                    j = 4*y
                    for a in range(i, i+4):
                        for b in range(j, j+4):
                            feature[x][y] += training_data[ino][a][b]
            features.append(feature)

        for ino in inos:    
            num_digits[training_y[ino]] += 1
            for x in range(7):
                for y in range(7):
                    prob_table_digits[training_y[ino]][x][y][features[ino][x][y]] += 1

        et = time.time()
        train_time = et-st
        df.loc[p]['Training_Time_Trail_'+str(trail)] = train_time


        test_accuracy = 0
        test_data, test_y = get_data(dtype="test")
        features_test = []
        for ino, label in enumerate(test_y):
            feature = [[0 for _ in range(7)] for _ in range(7)]
            for x in range(7):
                for y in range(7):
                    i = 4*x
                    j = 4*y
                    for a in range(i, i+4):
                        for b in range(j, j+4):
                            feature[x][y] += test_data[ino][a][b]
            features_test.append(feature)

        for ino in range(len(test_data)):
            probs  = [num_digits[i]/sum(num_digits) for i in range(10)]
            for x in range(7):
                for y in range(7):
                    for i in range(10):
                        factor = prob_table_digits[i][x][y][features_test[ino][x][y]]/sum(prob_table_digits[i][x][y]) if prob_table_digits[i][x][y][features_test[ino][x][y]] != 0 else 1/num_digits[i]
                        probs[i] *= factor
            max_label = 0
            max_prob = probs[0]
            for i in range(1, 10):
                if max_prob < probs[i]:
                    max_prob = probs[i]
                    max_label = i 

            if max_label == test_y[ino]:
                test_accuracy += 1
        
        
        test_accuracy /= len(test_data)

        #print("// ➔ Test Accuracy:", str(round(test_accuracy*100))+"%")
        pred_error = 1-test_accuracy
        df.loc[p]['Prediction_Error_Test_Trail_'+str(trail)] = pred_error

        validation_accuracy = 0
        validation_data, validation_y = get_data(dtype="validation")
        features_validation = []
        for ino, label in enumerate(validation_y):
            feature = [[0 for _ in range(7)] for _ in range(7)]
            for x in range(7):
                for y in range(7):
                    i = 4*x
                    j = 4*y
                    for a in range(i, i+4):
                        for b in range(j, j+4):
                            feature[x][y] += validation_data[ino][a][b]
            features_validation.append(feature)

        for ino in range(len(validation_data)):
            probs  = [num_digits[i]/sum(num_digits) for i in range(10)]
            for x in range(7):
                for y in range(7):
                    for i in range(10):
                        factor = prob_table_digits[i][x][y][features_validation[ino][x][y]]/sum(prob_table_digits[i][x][y]) if prob_table_digits[i][x][y][features_validation[ino][x][y]] != 0 else 1/num_digits[i]
                        probs[i] *= factor
            max_label = 0
            max_prob = probs[0]
            for i in range(1, 10):
                if max_prob < probs[i]:
                    max_prob = probs[i]
                    max_label = i 

            if max_label == validation_y[ino]:
                validation_accuracy += 1

        validation_accuracy /= len(validation_data)
        pred_error = 1-validation_accuracy
        #print("// ➔ validation Accuracy:", str(round(validation_accuracy*100))+"%")
        df.loc[p]['Prediction_Error_Validation_Trail_'+str(trail)] = pred_error
        print('Percentage',p,'Trail',trail, 'Ended')
        print()

        #print("////////////////////////////////")

Percentage 10 Trail 1 Started
Percentage 10 Trail 1 Ended

Percentage 10 Trail 2 Started
Percentage 10 Trail 2 Ended

Percentage 10 Trail 3 Started
Percentage 10 Trail 3 Ended

Percentage 10 Trail 4 Started
Percentage 10 Trail 4 Ended

Percentage 10 Trail 5 Started
Percentage 10 Trail 5 Ended

Percentage 10 Trail 6 Started
Percentage 10 Trail 6 Ended

Percentage 10 Trail 7 Started
Percentage 10 Trail 7 Ended

Percentage 10 Trail 8 Started
Percentage 10 Trail 8 Ended

Percentage 10 Trail 9 Started
Percentage 10 Trail 9 Ended

Percentage 10 Trail 10 Started
Percentage 10 Trail 10 Ended

Percentage 10 Trail 11 Started
Percentage 10 Trail 11 Ended

Percentage 10 Trail 12 Started
Percentage 10 Trail 12 Ended

Percentage 10 Trail 13 Started
Percentage 10 Trail 13 Ended

Percentage 10 Trail 14 Started
Percentage 10 Trail 14 Ended

Percentage 10 Trail 15 Started
Percentage 10 Trail 15 Ended

Percentage 10 Trail 16 Started
Percentage 10 Trail 16 Ended

Percentage 10 Trail 17 Started
Percentage 

Percentage 30 Trail 36 Ended

Percentage 30 Trail 37 Started
Percentage 30 Trail 37 Ended

Percentage 30 Trail 38 Started
Percentage 30 Trail 38 Ended

Percentage 30 Trail 39 Started
Percentage 30 Trail 39 Ended

Percentage 30 Trail 40 Started
Percentage 30 Trail 40 Ended

Percentage 30 Trail 41 Started
Percentage 30 Trail 41 Ended

Percentage 30 Trail 42 Started
Percentage 30 Trail 42 Ended

Percentage 30 Trail 43 Started
Percentage 30 Trail 43 Ended

Percentage 30 Trail 44 Started
Percentage 30 Trail 44 Ended

Percentage 30 Trail 45 Started
Percentage 30 Trail 45 Ended

Percentage 30 Trail 46 Started
Percentage 30 Trail 46 Ended

Percentage 30 Trail 47 Started
Percentage 30 Trail 47 Ended

Percentage 30 Trail 48 Started
Percentage 30 Trail 48 Ended

Percentage 30 Trail 49 Started
Percentage 30 Trail 49 Ended

Percentage 30 Trail 50 Started
Percentage 30 Trail 50 Ended

Percentage 40 Trail 1 Started
Percentage 40 Trail 1 Ended

Percentage 40 Trail 2 Started
Percentage 40 Trail 2 Ended

Percentage 60 Trail 22 Ended

Percentage 60 Trail 23 Started
Percentage 60 Trail 23 Ended

Percentage 60 Trail 24 Started
Percentage 60 Trail 24 Ended

Percentage 60 Trail 25 Started
Percentage 60 Trail 25 Ended

Percentage 60 Trail 26 Started
Percentage 60 Trail 26 Ended

Percentage 60 Trail 27 Started
Percentage 60 Trail 27 Ended

Percentage 60 Trail 28 Started
Percentage 60 Trail 28 Ended

Percentage 60 Trail 29 Started
Percentage 60 Trail 29 Ended

Percentage 60 Trail 30 Started
Percentage 60 Trail 30 Ended

Percentage 60 Trail 31 Started
Percentage 60 Trail 31 Ended

Percentage 60 Trail 32 Started
Percentage 60 Trail 32 Ended

Percentage 60 Trail 33 Started
Percentage 60 Trail 33 Ended

Percentage 60 Trail 34 Started
Percentage 60 Trail 34 Ended

Percentage 60 Trail 35 Started
Percentage 60 Trail 35 Ended

Percentage 60 Trail 36 Started
Percentage 60 Trail 36 Ended

Percentage 60 Trail 37 Started
Percentage 60 Trail 37 Ended

Percentage 60 Trail 38 Started
Percentage 60 Trail 38 E

Percentage 90 Trail 8 Ended

Percentage 90 Trail 9 Started
Percentage 90 Trail 9 Ended

Percentage 90 Trail 10 Started
Percentage 90 Trail 10 Ended

Percentage 90 Trail 11 Started
Percentage 90 Trail 11 Ended

Percentage 90 Trail 12 Started
Percentage 90 Trail 12 Ended

Percentage 90 Trail 13 Started
Percentage 90 Trail 13 Ended

Percentage 90 Trail 14 Started
Percentage 90 Trail 14 Ended

Percentage 90 Trail 15 Started
Percentage 90 Trail 15 Ended

Percentage 90 Trail 16 Started
Percentage 90 Trail 16 Ended

Percentage 90 Trail 17 Started
Percentage 90 Trail 17 Ended

Percentage 90 Trail 18 Started
Percentage 90 Trail 18 Ended

Percentage 90 Trail 19 Started
Percentage 90 Trail 19 Ended

Percentage 90 Trail 20 Started
Percentage 90 Trail 20 Ended

Percentage 90 Trail 21 Started
Percentage 90 Trail 21 Ended

Percentage 90 Trail 22 Started
Percentage 90 Trail 22 Ended

Percentage 90 Trail 23 Started
Percentage 90 Trail 23 Ended

Percentage 90 Trail 24 Started
Percentage 90 Trail 24 Ende

In [41]:
df

,Prediction_Error_Test_Trail_1,Prediction_Error_Test_Trail_2,Prediction_Error_Test_Trail_3,Prediction_Error_Test_Trail_4,Prediction_Error_Test_Trail_5,Prediction_Error_Test_Trail_6,Prediction_Error_Test_Trail_7,Prediction_Error_Test_Trail_8,Prediction_Error_Test_Trail_9,Prediction_Error_Test_Trail_10,...,Training_Time_Trail_41,Training_Time_Trail_42,Training_Time_Trail_43,Training_Time_Trail_44,Training_Time_Trail_45,Training_Time_Trail_46,Training_Time_Trail_47,Training_Time_Trail_48,Training_Time_Trail_49,Training_Time_Trail_50
Data%,,,,,,,,,,,,,,,,,,,,,
10,0.37,0.427,0.372,0.396,0.387,0.376,0.385,0.377,0.384,0.407,...,0.112656,0.118721,0.114239,0.114269,0.126187,0.116197,0.112224,0.120776,0.112698,0.115693
20,0.307,0.314,0.306,0.309,0.296,0.307,0.317,0.31,0.302,0.314,...,0.240393,0.236368,0.246373,0.233345,0.23138,0.309436,0.227895,0.242351,0.23242,0.230917
30,0.278,0.282,0.289,0.282,0.302,0.268,0.289,0.281,0.28,0.283,...,0.359075,0.348042,0.352057,0.348105,0.352091,0.352085,0.353056,0.346074,0.439335,0.359514
40,0.268,0.29,0.27,0.288,0.272,0.274,0.261,0.284,0.278,0.273,...,0.465724,0.460316,0.474277,0.470741,0.462282,0.478752,0.474699,0.468711,0.464987,0.4608
50,0.273,0.267,0.265,0.267,0.276,0.275,0.274,0.258,0.247,0.27,...,0.583417,0.638103,0.578456,0.594445,0.594299,0.572793,0.586431,0.577928,0.585216,0.578694
60,0.269,0.254,0.269,0.253,0.261,0.257,0.255,0.255,0.259,0.257,...,0.690975,0.780022,0.712197,0.713109,0.730667,0.704232,0.690537,0.699818,0.765753,0.717102
70,0.261,0.254,0.25,0.252,0.263,0.262,0.263,0.259,0.255,0.26,...,0.9181,0.834317,0.821114,0.818777,0.893137,0.810916,0.82733,0.823952,0.903664,0.823352
80,0.254,0.262,0.25,0.254,0.249,0.252,0.25,0.247,0.259,0.259,...,0.939578,1.02514,0.960938,0.934471,0.954045,0.956049,1.03804,0.950995,0.95174,0.925977
90,0.248,0.252,0.258,0.249,0.25,0.253,0.259,0.259,0.253,0.256,...,1.07368,1.06025,1.06455,1.21379,1.22826,1.10782,1.08015,1.061,1.15162,1.15094


In [45]:
df.to_csv('Digits_Bayes.csv')

In [37]:
df.loc[10]

Prediction_Error_Test_Trail_1       0.365
Prediction_Error_Test_Trail_2       0.389
Prediction_Error_Test_Trail_3       0.375
Prediction_Error_Test_Trail_4       0.375
Prediction_Error_Test_Trail_5       0.409
                                   ...   
Training_Time_Trail_46           0.111699
Training_Time_Trail_47           0.124666
Training_Time_Trail_48           0.114662
Training_Time_Trail_49           0.124634
Training_Time_Trail_50           0.121675
Name: 10, Length: 150, dtype: object

In [43]:
df['Average_Training_Time'] = df.iloc[:,101:151].mean(axis=1)
df['Average_Prediction_Error_Test'] = df.iloc[:,:51].mean(axis=1)
df['Average_Prediction_Error_Validation'] = df.iloc[:,51:101].mean(axis=1)
df['Standard_Deviation_Test_Error'] = df.iloc[:, :51].std(axis=1)
df['Standard_Deviation_Validation_Error'] = df.iloc[:,51:101].std(axis=1)
df

,Prediction_Error_Test_Trail_1,Prediction_Error_Test_Trail_2,Prediction_Error_Test_Trail_3,Prediction_Error_Test_Trail_4,Prediction_Error_Test_Trail_5,Prediction_Error_Test_Trail_6,Prediction_Error_Test_Trail_7,Prediction_Error_Test_Trail_8,Prediction_Error_Test_Trail_9,Prediction_Error_Test_Trail_10,...,Training_Time_Trail_46,Training_Time_Trail_47,Training_Time_Trail_48,Training_Time_Trail_49,Training_Time_Trail_50,Average_Training_Time,Average_Prediction_Error_Test,Average_Prediction_Error_Validation,Standard_Deviation_Test_Error,Standard_Deviation_Validation_Error
Data%,,,,,,,,,,,,,,,,,,,,,
10,0.37,0.427,0.372,0.396,0.387,0.376,0.385,0.377,0.384,0.407,...,0.116197,0.112224,0.120776,0.112698,0.115693,0.119302,0.378647,0.346394,0.016913,0.035455
20,0.307,0.314,0.306,0.309,0.296,0.307,0.317,0.31,0.302,0.314,...,0.309436,0.227895,0.242351,0.23242,0.230917,0.239736,0.310216,0.279837,0.011915,0.010459
30,0.278,0.282,0.289,0.282,0.302,0.268,0.289,0.281,0.28,0.283,...,0.352085,0.353056,0.346074,0.439335,0.359514,0.367845,0.284961,0.253751,0.010072,0.016507
40,0.268,0.29,0.27,0.288,0.272,0.274,0.261,0.284,0.278,0.273,...,0.478752,0.474699,0.468711,0.464987,0.4608,0.472587,0.272314,0.243167,0.009348,0.033173
50,0.273,0.267,0.265,0.267,0.276,0.275,0.274,0.258,0.247,0.27,...,0.572793,0.586431,0.577928,0.585216,0.578694,0.592355,0.264510,0.237480,0.008564,0.052291
60,0.269,0.254,0.269,0.253,0.261,0.257,0.255,0.255,0.259,0.257,...,0.704232,0.690537,0.699818,0.765753,0.717102,0.717931,0.258882,0.236258,0.007888,0.076825
70,0.261,0.254,0.25,0.252,0.263,0.262,0.263,0.259,0.255,0.26,...,0.810916,0.82733,0.823952,0.903664,0.823352,0.838041,0.256098,0.234360,0.008672,0.083263
80,0.254,0.262,0.25,0.254,0.249,0.252,0.25,0.247,0.259,0.259,...,0.956049,1.03804,0.950995,0.95174,0.925977,0.956703,0.253451,0.234329,0.007651,0.103875
90,0.248,0.252,0.258,0.249,0.25,0.253,0.259,0.259,0.253,0.256,...,1.10782,1.08015,1.061,1.15162,1.15094,1.086642,0.250863,0.236183,0.006148,0.131505


In [44]:
df

,Prediction_Error_Test_Trail_1,Prediction_Error_Test_Trail_2,Prediction_Error_Test_Trail_3,Prediction_Error_Test_Trail_4,Prediction_Error_Test_Trail_5,Prediction_Error_Test_Trail_6,Prediction_Error_Test_Trail_7,Prediction_Error_Test_Trail_8,Prediction_Error_Test_Trail_9,Prediction_Error_Test_Trail_10,...,Training_Time_Trail_46,Training_Time_Trail_47,Training_Time_Trail_48,Training_Time_Trail_49,Training_Time_Trail_50,Average_Training_Time,Average_Prediction_Error_Test,Average_Prediction_Error_Validation,Standard_Deviation_Test_Error,Standard_Deviation_Validation_Error
Data%,,,,,,,,,,,,,,,,,,,,,
10,0.37,0.427,0.372,0.396,0.387,0.376,0.385,0.377,0.384,0.407,...,0.116197,0.112224,0.120776,0.112698,0.115693,0.119302,0.378647,0.346394,0.016913,0.035455
20,0.307,0.314,0.306,0.309,0.296,0.307,0.317,0.31,0.302,0.314,...,0.309436,0.227895,0.242351,0.23242,0.230917,0.239736,0.310216,0.279837,0.011915,0.010459
30,0.278,0.282,0.289,0.282,0.302,0.268,0.289,0.281,0.28,0.283,...,0.352085,0.353056,0.346074,0.439335,0.359514,0.367845,0.284961,0.253751,0.010072,0.016507
40,0.268,0.29,0.27,0.288,0.272,0.274,0.261,0.284,0.278,0.273,...,0.478752,0.474699,0.468711,0.464987,0.4608,0.472587,0.272314,0.243167,0.009348,0.033173
50,0.273,0.267,0.265,0.267,0.276,0.275,0.274,0.258,0.247,0.27,...,0.572793,0.586431,0.577928,0.585216,0.578694,0.592355,0.264510,0.237480,0.008564,0.052291
60,0.269,0.254,0.269,0.253,0.261,0.257,0.255,0.255,0.259,0.257,...,0.704232,0.690537,0.699818,0.765753,0.717102,0.717931,0.258882,0.236258,0.007888,0.076825
70,0.261,0.254,0.25,0.252,0.263,0.262,0.263,0.259,0.255,0.26,...,0.810916,0.82733,0.823952,0.903664,0.823352,0.838041,0.256098,0.234360,0.008672,0.083263
80,0.254,0.262,0.25,0.254,0.249,0.252,0.25,0.247,0.259,0.259,...,0.956049,1.03804,0.950995,0.95174,0.925977,0.956703,0.253451,0.234329,0.007651,0.103875
90,0.248,0.252,0.258,0.249,0.25,0.253,0.259,0.259,0.253,0.256,...,1.10782,1.08015,1.061,1.15162,1.15094,1.086642,0.250863,0.236183,0.006148,0.131505
